In [1]:
import csv
#import cv2

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import PIL
import numpy as np
import pandas as pd
#import matplotlib as mpl
import random
#mpl.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, Lambda, Cropping2D
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten
from scipy import misc
from scipy.ndimage import rotate
from skimage import transform
from skimage.transform import warp, SimilarityTransform, AffineTransform
from skimage import exposure
import os 
from os import listdir
import glob
import pandas as pd
from io import BytesIO
import base64


Using TensorFlow backend.


In [2]:
from PIL import Image

In [3]:
import os 
from os import listdir
import glob
import pandas as pd
from io import BytesIO
import base64

In [4]:
os.chdir("30D_1/New directory/Img/")
filenames = [img for img in glob.glob("*.png")]
import re
tokenize = re.compile(r'(\d+)|(\D+)').findall
def natural_sortkey(string):          
    return tuple(int(num) if num else alpha for num, alpha in tokenize(string))

files = sorted(filenames, key=natural_sortkey)

files

['30D_9_1.png',
 '30D_9_2.png',
 '30D_9_3.png',
 '30D_9_4.png',
 '30D_9_5.png',
 '30D_9_6.png',
 '30D_9_7.png',
 '30D_9_8.png',
 '30D_9_9.png',
 '30D_9_10.png',
 '30D_9_11.png',
 '30D_9_12.png',
 '30D_9_13.png',
 '30D_9_14.png',
 '30D_9_15.png',
 '30D_9_16.png',
 '30D_9_17.png',
 '30D_9_18.png',
 '30D_9_19.png',
 '30D_9_20.png',
 '30D_9_21.png',
 '30D_9_22.png',
 '30D_9_23.png',
 '30D_9_24.png',
 '30D_9_25.png',
 '30D_9_26.png',
 '30D_9_27.png',
 '30D_9_28.png',
 '30D_9_29.png',
 '30D_9_30.png',
 '30D_9_31.png',
 '30D_9_32.png',
 '30D_9_33.png',
 '30D_9_34.png',
 '30D_9_35.png',
 '30D_9_36.png',
 '30D_9_37.png',
 '30D_9_38.png',
 '30D_9_39.png',
 '30D_9_40.png',
 '30D_9_41.png',
 '30D_9_42.png',
 '30D_9_43.png',
 '30D_9_44.png',
 '30D_9_45.png',
 '30D_9_46.png',
 '30D_9_47.png',
 '30D_9_48.png',
 '30D_9_49.png',
 '30D_9_50.png',
 '30D_9_51.png',
 '30D_9_52.png',
 '30D_9_53.png',
 '30D_9_54.png',
 '30D_9_55.png',
 '30D_9_56.png',
 '30D_9_57.png',
 '30D_9_58.png',
 '30D_9_59.png',
 '30D_

In [5]:
labels = pd.read_csv("../30D_9.csv")

In [6]:
image = mpimg.imread(files[0])

In [7]:
images = []
for i in range(len(files)):
    images.append(mpimg.imread(files[i]))

In [8]:
images[0].shape

(231, 231, 4)

In [9]:
labels["strain"]

0      0.000377
1      0.000592
2      0.000774
3      0.000891
4      0.001091
         ...   
558    0.120700
559    0.120900
560    0.121200
561    0.121500
562    0.121700
Name: strain, Length: 563, dtype: float64

In [10]:
X_train = np.array(images)

y = np.array(labels["stress"]).max()
y_train = np.array(labels["stress"])/y

In [11]:
y_train

array([0.35192937, 0.55140615, 0.71877044, 0.82603009, 1.        ,
       0.81425768, 0.65533028, 0.51098757, 0.42910399, 0.37311969,
       0.39489863, 0.41066056, 0.42393721, 0.42740353, 0.44100719,
       0.47730543, 0.46664487, 0.44571615, 0.43597122, 0.43989536,
       0.4559843 , 0.45448005, 0.4824722 , 0.47848267, 0.48423806,
       0.48940484, 0.49862655, 0.50124264, 0.49372139, 0.49895356,
       0.49483322, 0.49529104, 0.47580118, 0.47272727, 0.47037279,
       0.45676913, 0.44290386, 0.43283192, 0.45271419, 0.42537606,
       0.39947678, 0.39156311, 0.38966645, 0.39306736, 0.39646828,
       0.39568345, 0.39587966, 0.41661216, 0.41909745, 0.42661871,
       0.42812296, 0.42583388, 0.43544801, 0.42465664, 0.44480052,
       0.44336167, 0.43485939, 0.43649444, 0.43950294, 0.44741661,
       0.44362328, 0.44460432, 0.4500327 , 0.45369523, 0.44480052,
       0.44290386, 0.44767822, 0.44316547, 0.43858731, 0.4412034 ,
       0.44604317, 0.44983649, 0.4440157 , 0.45689993, 0.45179

In [12]:
model = Sequential()

#The architecture is similar to NVIDIA architecture. It starts with convolutional layers that have 
#24, 36 and 48 , 5x5 filters respectively in the first 3 layers and a stride of 2 in both the directions.
#These layers are followed by two convolutional layers each having 64 3x3 filters with strides of 1
#in both the directions. Exponential Linear Unit activation function is used in all the five convolutional
#layers. A maxpooling layer follows. The layer is flattened and fed into four fully connected layers with
#first two having ELU activation. Dropout is applied in each layer after the first one with a probability of
#retaining the weights equal to 0.5. It was also realized that maxpooling layers take a long time
model.add(Convolution2D(24,(2,2), padding = 'valid', strides = (2,2), activation='elu'))
model.add(Convolution2D(36,(2,2), padding = 'valid', strides = (2,2), activation='elu'))
model.add(Convolution2D(48,(3,3), padding = 'valid', strides = (2,2), activation='elu'))
model.add(Convolution2D(64,(3,3), padding = 'valid', strides = (1,1), activation='elu'))
model.add(Convolution2D(64,(3,3), padding = 'valid', strides = (1,1), activation='elu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(50, activation = 'elu'))
model.add(Dropout(0.5))
model.add(Dense(25))
model.add(Dropout(0.5))
model.add(Dense(1))

In [13]:
#sgd = tf.keras.optimizers.SGD()
sgd = optimizers.SGD(lr=0.00001)
#model.compile(loss='mean_squared_error', optimizer=sgd)
model.compile(loss = 'mse', optimizer = sgd)    #mse is used here instead of a cross entropy function because it is a regression and not a classification
hist = model.fit(X_train, y_train, batch_size = 20, validation_split = 0.25, shuffle = True, epochs = 30)


#model.save('model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 422 samples, validate on 141 samples
Epoch 1/30
422/422 [==============================] - 21s 50ms/step - loss: 0.7932 - val_loss: 0.0580
Epoch 2/30
422/422 [==============================] - 19s 45ms/step - loss: 0.5329 - val_loss: 0.0393
Epoch 3/30
422/422 [==============================] - 19s 44ms/step - loss: 0.4931 - val_loss: 0.0171
Epoch 4/30
422/422 [==============================] - 18s 43ms/step - loss: 0.4400 - val_loss: 0.0158
Epoch 5/30
422/422 [==============================] - 18s 44ms/step - loss: 0.3746 - val_loss: 0.0128
Epoch 6/30
422/422 [==============================] - 18s 43ms/step - loss: 0.3400 - val_loss: 0.0077
Epoch 7/30
422/422 [==============================] - 18s 44ms/step - loss: 0.3193 - val_loss: 0.0084
Epoch 8/30
422/422 [==============================] - 18s 43ms/step - loss: 0.2779 - val_loss: 0.0099
Epoch 9/30
422/422

In [ ]:
#print(hist.history)
train_loss = hist.history['loss']
val_loss   = hist.history['val_loss']

predicted_strain = []
l = []
for i in range(len(files)):
    image = images[i]
    image = np.expand_dims(image, axis=0)
    k = (model.predict(image))*y
    l.append(k[0][0])
    
    
plt.figure(figsize=(12,7))

plt.xlabel("Strain",fontsize = 14)
plt.ylabel("Stress (x10 MPa)",fontsize = 14)

j = np.array(labels["strain"][30:300])
plt.plot( j,l[30:300], 'r*', label='Predicted Stress')
k = np.array(labels["stress"][30:300])
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.plot(j,k,"g",label = "Actual Stress")
plt.legend(prop={"size":12})
plt.title('Actual and predicted value of Stress',fontsize = 20)




In [ ]:
#print(hist.history)
train_loss = hist.history['loss'][1:]
val_loss   = hist.history['val_loss'][1:]

plt.figure(figsize=(7,7))
epochs = range(len(val_loss))
plt.plot(epochs, train_loss, 'r', label='Training loss')
plt.xlabel("No. of Epochs",fontsize = 14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel("Fraction",fontsize = 14)
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss',fontsize = 20)
plt.legend(prop={"size":12})

plt.figure(figsize=(10,10))

In [18]:
model.save("model_24_5.h5")